In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
##### Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../WeatherPy/OutputData/CleanCityData.csv")
weather_df.head()

,Unnamed: 0,City,Country,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Coolum Beach,AU,100,97,-26.53,153.10,63.00,1.99
1,1,Jamestown,US,1,45,42.10,-79.24,88.00,2.21
2,2,Busselton,AU,87,68,-33.65,115.33,46.99,7.83
3,3,Lexington,US,40,69,37.99,-84.48,82.99,13.87
4,4,Paraty,BR,20,73,-23.22,-44.71,71.60,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 10), zoom_level=1.75)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                 max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_df = weather_df.loc[(weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)].dropna()

narrow_df

,Unnamed: 0,City,Country,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
36,36,Maragogi,BR,0,85,-9.01,-35.22,76.28,5.88
43,43,Caravelas,BR,0,91,-17.71,-39.25,72.16,4.52
49,49,Alta Floresta,BR,0,74,-9.88,-56.09,76.19,3.71
194,194,Lodja,CD,0,60,-3.48,23.43,74.28,1.28
345,345,Taguatinga,BR,0,53,-12.39,-46.57,76.55,4.90
347,347,Kumluca,TR,0,48,36.37,30.29,73.58,2.73
360,360,Dawlatabad,AF,0,20,36.41,64.91,73.99,4.14
443,443,Beyneu,KZ,0,20,45.32,55.20,79.74,9.17
451,451,Morondava,MG,0,71,-20.28,44.28,70.30,5.12
468,468,Tura,IN,0,74,25.52,90.22,77.49,2.53


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_df[["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

C:\Users\mjbgo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Lat,Lng,Hotel Name
36,Maragogi,BR,-9.01,-35.22,
43,Caravelas,BR,-17.71,-39.25,
49,Alta Floresta,BR,-9.88,-56.09,
194,Lodja,CD,-3.48,23.43,
345,Taguatinga,BR,-12.39,-46.57,
347,Kumluca,TR,36.37,30.29,
360,Dawlatabad,AF,36.41,64.91,
443,Beyneu,KZ,45.32,55.20,
451,Morondava,MG,-20.28,44.28,
468,Tura,IN,25.52,90.22,


In [7]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key}


for index,row in hotel_df.iterrows():
    # get lat, lng from df
    lng = row["Lng"]
    lat = row["Lat"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_name = requests.get(base_url, params=params)
    

    # convert to json
    hotel_name = hotel_name.json()
    
    # get hotel name and put it into the data frame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        pass
    
#Print dataFrame with hotel names        
hotel_df

C:\Users\mjbgo\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Hotel Name
36,Maragogi,BR,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
43,Caravelas,BR,-17.71,-39.25,Mar Doce De Abrolhos
49,Alta Floresta,BR,-9.88,-56.09,Hotel Mandino
194,Lodja,CD,-3.48,23.43,
345,Taguatinga,BR,-12.39,-46.57,
347,Kumluca,TR,36.37,30.29,TurkuazKöy
360,Dawlatabad,AF,36.41,64.91,
443,Beyneu,KZ,45.32,55.20,"HOTEL ""AKNUR"""
451,Morondava,MG,-20.28,44.28,Kimony Resort Hotel
468,Tura,IN,25.52,90.22,Hotel Polo Orchid


In [8]:
# Remove rows with empty cells
cleanhotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")].set_index("City").reset_index()
cleanhotel_df

,City,Country,Lat,Lng,Hotel Name
0,Maragogi,BR,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
1,Caravelas,BR,-17.71,-39.25,Mar Doce De Abrolhos
2,Alta Floresta,BR,-9.88,-56.09,Hotel Mandino
3,Kumluca,TR,36.37,30.29,TurkuazKöy
4,Beyneu,KZ,45.32,55.20,"HOTEL ""AKNUR"""
5,Morondava,MG,-20.28,44.28,Kimony Resort Hotel
6,Tura,IN,25.52,90.22,Hotel Polo Orchid


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer for hotels on top of heat map
fig = gmaps.figure(center=(0, 10), zoom_level=1.75)
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(markers)
fig.add_layer(heat_layer)

fig
# Display Map

Figure(layout=FigureLayout(height='420px'))